<a href="https://colab.research.google.com/github/Malanidhruv/quant-projects/blob/main/Fno_considering_3_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2022, 1, 1)  # Start from a longer time frame
    to_datetime = datetime.now()  # To now
    interval = "D"  # ["1", "D"]

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Find local maxima and minima
    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]

    # Global maxima and minima
    global_maxima_index = np.argmax(close_prices)
    global_minima_index = np.argmin(close_prices)

    # Define the current price (example: last close price)
    current_price = df['close'].iloc[-1]

    # Define threshold (e.g., within 5% of the extrema)
    threshold = 0.05

    # Determine if current price is near any local or global minima or maxima
    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    # Check proximity to local minima
    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)

    # Check proximity to global minima
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    # Calculate the moving average of the volume
    df['volume_ma'] = df['volume'].rolling(window=20).mean()

    # Volume threshold (e.g., 1.5 times the average volume)
    volume_threshold = 1.5 * df['volume'].mean()

    # Enhanced signal determination using volume
    if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
        signal = "BUY"
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            support_level = close_prices[nearest_local_minima_index]
            support_volume = df.loc[nearest_local_minima_index, 'volume']
        else:
            support_level = close_prices[global_minima_index]
            support_volume = df.loc[global_minima_index, 'volume']
        if support_volume < volume_threshold:
            signal = None  # Downgrade to None if volume is not confirming
    else:
        signal = None

    return token, current_price, signal, support_level if signal == "BUY" else None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    result = analyze_stock(alice, token)
    if result[2] == "BUY":
        buy_signals.append((result[0], result[1], result[3]))

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)


Enter your user ID: 1141826
Enter your API key: 9eMrXt1qMSQbkiFRlcSbV2B5COKfKgXrWp0QMyTnWftE1fhu8J7EmhkUQ5t4L9uxuVDXVDxWczijxYK4YJmTuAyHafa80ebLen80gDUSc7TodQ5k7EHDys9IdPM3QegY
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
NESTLEIND-EQ    17963      2597.85         2593.20        
TCS-EQ          11536      4302.40         4169.20        
BHARTIARTL-EQ   10604      1461.75         1444.05        
BAJFINANCE-EQ   317        6932.30         6931.75        
--------------------------------------------------


In [8]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, time
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def get_today_data(alice, token):
    current_date = datetime.now()
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime.combine(current_date, time(9, 15))
    to_datetime = datetime.combine(current_date, time(15, 30))
    interval = "1"  # Minute-wise data

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Convert the list of dictionaries to a DataFrame
    df_today = pd.DataFrame(historical_data)

    # Check if 'datetime' key exists
    if 'datetime' in df_today.columns:
        df_today['datetime'] = pd.to_datetime(df_today['datetime'])
    else:
        raise KeyError("'datetime' key not found in historical data")

    start_time = datetime.combine(current_date, time(9, 15))
    end_time = datetime.combine(current_date, time(15, 15, 59))
    filtered_data = df_today[(df_today['datetime'] >= start_time) &
                             (df_today['datetime'] <= end_time)]

    cumulative_volume = filtered_data['volume'].sum()

    new_daily_data = {
        'datetime': [end_time],
        'open': [filtered_data.iloc[0]['open']],
        'high': [filtered_data['high'].max()],
        'low': [filtered_data['low'].min()],
        'close': [filtered_data.iloc[-1]['close']],
        'volume': [cumulative_volume]
    }
    return pd.DataFrame(new_daily_data)

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2022, 1, 1)
    to_datetime = datetime.now()
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Add today's data
    today_data = get_today_data(alice, token)
    df = pd.concat([df, today_data], ignore_index=True)

    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]
    global_maxima_index = np.argmax(close_prices)
    global_minima_index = np.argmin(close_prices)
    current_price = df['close'].iloc[-1]

    threshold = 0.05

    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    df['volume_ma'] = df['volume'].rolling(window=20).mean()
    volume_threshold = 1.5 * df['volume'].mean()

    if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
        signal = "BUY"
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            support_level = close_prices[nearest_local_minima_index]
            support_volume = df.loc[nearest_local_minima_index, 'volume']
        else:
            support_level = close_prices[global_minima_index]
            support_volume = df.loc[global_minima_index, 'volume']
        if support_volume < volume_threshold:
            signal = None
    else:
        signal = None

    return token, current_price, signal, support_level if signal == "BUY" else None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    try:
        result = analyze_stock(alice, token)
        if result[2] == "BUY":
            buy_signals.append((result[0], result[1], result[3]))
    except Exception as e:
        print(f"Error analyzing token {token}: {e}")

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)


Enter your user ID: 1141826
Enter your API key: 9eMrXt1qMSQbkiFRlcSbV2B5COKfKgXrWp0QMyTnWftE1fhu8J7EmhkUQ5t4L9uxuVDXVDxWczijxYK4YJmTuAyHafa80ebLen80gDUSc7TodQ5k7EHDys9IdPM3QegY
Error analyzing token 2031: If using all scalar values, you must pass an index
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
GRASIM-EQ       1232       2804.05         2803.75        
NESTLEIND-EQ    17963      2588.25         2585.60        
TCS-EQ          11536      4288.75         4169.20        
--------------------------------------------------
